In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

### Scanner Tests for Exception

In [ ]:
import SC
def scanString(src):
    SC.init(src); syms = []
    while SC.sym != SC.EOF:
        syms.append(('INDENT' if SC.sym == SC.INDENT else \
                     'DEDENT' if SC.sym == SC.DEDENT else \
                     'IDENT' if SC.sym == SC.IDENT else SC.sym, SC.newline))
        SC.getSym()
    return syms

#### Parsing Explicit Exceptions

In [ ]:
scanString("""

program p

  if a then
    writeln()
  else
    writeln()
  if a then writeln() else writeln()
""")

In [ ]:
scanString("""

program p

  if a then
    throw
  else
    throw
  if a then writeln() else writeln()
""")

In [ ]:
scanString("""

program p

  if a then
    throw exception('Just a test')
  else
    throw
  if a then throw exception('Just a test') else writeln()
""")

### Compiler Tests for Exception

In [ ]:
# "Throw" can be compiled，but the WebAssembly Code is incorrect
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else r := x
program equationsolver
    var a: integer
        a ← sqrt(2)
        write(a)
""")

In [ ]:
!wat2wasm test.wat

#### Zero Division Exception

In [ ]:
compileString("""

program DividedByZero
    var a, b: integer
        a := (34 * 5 - 170)
        b := (99 div a)
        write(a)
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [ ]:
compileString("""

program DividedByZero
    var b: integer
        b := (99 div 0)
        write(b)
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

#### Access Before Initialization Exception

In [ ]:
compileString("""

program AccessBeforeInitialization
    a := 5
    var a: integer
    
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

#### Wrong Type Exception

In [ ]:
compileString("""

program WrongType
    a := 5
    var a: boolean
    
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

#### Variable Name Not Defined Exception

In [ ]:
compileString("""

program bIsNotDefined
    var a: boolean
    b := 5
    write(b)
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

#### Index Out of Bounds Exception

In [ ]:
compileString("""

var x: [2 .. 4] → integer
program IndexOutOfBounds
  x[7] := 5
  
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [ ]:
compileString("""

var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[1] := 1 
    x[2] := 1
    x[6] := 1
  
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [ ]:
compileString("""

var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    i := 7
    write(x[i])
  
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

#### Syntax Error Exception

In [ ]:
compileString("""

program missingProcedure
  var b: boolean
    b ← integer()
    
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [ ]:
compileString("""

program incompatibleCall
  var b: boolean
    b ← read()
    
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else
        r := 0;
        while (r + 1) × (r + 1) ≤ y do
            r := r + 1
        x := r
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c, x, y: integer
    var done: boolean
        done := false
        while ¬done do
            try
                a ← read(); b ← read(); c ← read()
                x, y ← quadraticsolution(a, b, c)
                write(x); write(y)
            catch
                done := true
""")
